In [1]:
import sys
from sys import exit
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))

In [2]:
import codecs
import pandas as pd
import numpy as np
from bulletin.services.metabase import Metabase

mb = Metabase()

pd.set_option('display.max_columns', None)


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------

Cookie:'8d43f********-****-****-****-************5aece'

saved_queries: ['diario', 'old_diario', 'all_notifica'] 



In [3]:
notificacao, information_schema, constraint = mb.download_tables()

Success code 200
Download finish, time elapsed: 0:00:00.432689
downloaded shape (179, 3)

Success code 200
Download finish, time elapsed: 0:00:00.369113
downloaded shape (3933, 3)

Success code 200
Download finish, time elapsed: 0:00:01.014853
downloaded shape (219, 5)

Download tables time elepsed:00:00:02


In [4]:
# notificacao.loc[notificacao['foreign_table_name']=='sim_nao', 'udt_name'] = 'bool'
# notificacao.loc[notificacao['foreign_table_name']=='sim_nao', ['constraint_name', 'foreign_table_name', 'foreign_column_name']] = ''
notificacao.to_csv('notificacao_constraint.csv', index=False)
notificacao

,table_name,column_name,udt_name,constraint_name,foreign_table_name,foreign_column_name
0,notificacao,id,int4,,,
1,notificacao,data_notificacao,timestamp,,,
2,notificacao,possui_cpf,int4,possui_cpf,sim_nao,id
3,notificacao,tipo_paciente,int4,tipo_paciente,tipo_paciente,id
4,notificacao,pais_residencia,varchar,pais_residencia,pais,co_pais
...,...,...,...,...,...,...
174,notificacao,lote_covid19_1a,varchar,,,
175,notificacao,data_covid19_2a,date,,,
176,notificacao,lote_covid19_2a,varchar,,,
177,notificacao,vacina_influenza,int4,,,


In [5]:
exit()

SystemExit: 

/home/vinnylg/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
constraint = constraint.loc[constraint['foreign_table_name']!='sim_nao'] 
constraint

In [ ]:
with codecs.open('tables.dbml',"w","utf-8-sig") as out:

    for table_name, table in information_schema.groupby('table_name'):
        if any(table_name in n for n in notificacao['foreign_table_name'].values):
            out.write(f"Table {table_name}" + " {\n")
            for column_name, udt_name in zip(table['column_name'],table['udt_name']):
                if column_name in constraint.loc[constraint['table_name'] == table_name, 'foreign_column_name'].values:
                    out.write(f"\t {column_name} {udt_name} [pk]\n")
                    notificacao.loc[notificacao['foreign_table_name']==table_name,'ref'] = f"[ref: - {table_name}.{column_name}]"
                else:
                    out.write(f"\t {column_name} {udt_name}\n")
            out.write("}\n\n")


    out.write(f"Table notificacao" + " {\n")
    for column_name, udt_name,ref in zip(notificacao['column_name'],notificacao['udt_name'],notificacao['ref']):
        out.write(f"\t {column_name} {udt_name} {ref if ref != None else ''}\n")
    out.write("}\n\n")